In [1]:
%reload_ext autoreload
#%load_ext autoreload
%autoreload 1
%matplotlib inline

import datetime
#import pandas as pd
import modin.pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from tqdm import tqdm


#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import os
import gc

os.chdir("T:\\laupodteam\\AIOS\\Bram\\notebooks\\code_dev\\celldyn_embedder\\sandbox")
#os.chdir("/media/UMCU/notebooks/code_dev/celldyn_embedder/sandbox")
sns.set_style("whitegrid")

from celldyn_qc import QcControl
from ctransformers import CellDynTrans

write_to_disk = False
read_from_disk = True

import ray
ray.init()

2022-08-26 16:14:19,995	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.10', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:62673', 'raylet_socket_name': 'tcp://127.0.0.1:61688', 'webui_url': '127.0.0.1:8265', 'session_dir': 'C:\\Users\\bes3\\AppData\\Local\\Temp\\ray\\session_2022-08-26_16-14-17_134492_15236', 'metrics_export_port': 64351, 'gcs_address': '127.0.0.1:61808', 'address': '127.0.0.1:61808', 'node_id': 'de199d594106fd64fe61887222154f7a6cd2a3ec5339bb22bc5bdc41'})

* filter based on group flags
* filter based on suspect flags
* filter based on range
* apply scaling
* correct for diurnal rythm
* impute

In [3]:
def sigmoid(x):
    #return np.tanh(x)
    return 1/(1+np.exp(-x))

def sigmoid_log(x):
    return sigmoid(np.log(x+1))

There are no special NaN-modes; the NaN's are the result of running in non-reti modus, the affected measurements are:

`
'c_b_retc',
'c_b_pretc',
'c_b_irf',
'c_b_pHPO',
'c_b_pHPR',
'c_b_HDW',
'c_b_MCVr',
'c_b_MCHr',
'c_b_MCHCr',
'c_b_rbcimn',
'c_b_rbcicv',
'c_b_rbcfmn',
'c_b_rbcfcv'
`

In [4]:
celldyn = pd.read_feather("T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn.feather")
celldyn.columns = celldyn.columns.str.lower()

In [5]:
gc.collect()

24

In [6]:

meta_cols = ['asp_dt', 'analyzer', 'studyId_Alle_celldyn', 
             'time_to_measurement', 'gender', 'age' , 'afname_dt',
             'day_of_year', 'week_of_year', 'year', 'first_day',
              'draw_hour', 'meas_hour']
count_columns = [c for c in celldyn.columns if 'c_cnt' in c]
meas_columns = [c for c in celldyn.columns if 'c_b' in c]+['plt']
mode_columns = [c for c in celldyn.columns if 'c_m' in c]
susp_columns = [c for c in celldyn.columns if 'c_s' in c]
alert_columns = [c for c in celldyn.columns if 'alrt' in c]
fail_columns = [c for c in celldyn.columns if 'fail' in c]
all_cols  = meta_cols+meas_columns+mode_columns+susp_columns+alert_columns+count_columns+fail_columns
other_cols = [c for c in celldyn.columns if c not in all_cols]

celldyn['gender']= celldyn.gender.str.decode(encoding='latin1')
celldyn['age'] = celldyn.age.astype(int)
celldyn['meas_hour'] = celldyn['asp_dt'].dt.hour#   apply(lambda x: x.hour)
celldyn['draw_hour'] = celldyn['afname_dt'].dt.hour # .apply(lambda x: x.hour)
celldyn['time_to_measurement'] = (celldyn.asp_dt-celldyn.afname_dt).dt.total_seconds()
celldyn['time_to_measurement'] = celldyn['time_to_measurement']//3600

cond = (celldyn.time_to_measurement<10) & (celldyn.time_to_measurement>-4)
celldyn = celldyn[cond]

def next_weekday(d, weekday):
    days_diff = weekday - d.weekday()
    return (d + datetime.timedelta(days_diff))\
    .replace(hour=0, minute=0, second=0, microsecond=0)

celldyn['day_of_year'] = np.nan
celldyn['day_of_year'] = celldyn.loc[celldyn.c_mode_xlyse==0, 'asp_dt']\
                            .apply(lambda x: x.dayofyear)
celldyn['week_of_year'] = celldyn.loc[celldyn.c_mode_xlyse==0, 'asp_dt']\
                            .apply(lambda x: x.weekofyear)

celldyn['year'] = celldyn.loc[celldyn.c_mode_xlyse==0, 'asp_dt'].apply(lambda x: x.year)
celldyn['first_day'] = celldyn.afname_dt.apply(lambda x: next_weekday(x, 0))

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [7]:
celldyn['mode_combos'] = celldyn[mode_columns]\
                        .apply(lambda x: ",".join([str(_x) for _x in x]),\
                         axis=1)

nan_modes = {}
for c in tqdm(celldyn['mode_combos'].unique()):
    condition = celldyn['mode_combos'] == c
    meas_df=celldyn.loc[condition, meas_columns]
    N = meas_df.shape[0]
    named_c = ",".join([f"{mode_columns[i]}:{str(int(float(v)))}"
                 for i, v in enumerate(c.split(","))])
    nan_list = meas_df.columns[meas_df.isna().sum()/N>=0.8].tolist()
    if len(nan_list)>0:
        nan_modes[named_c] = nan_list
        
reti_mode_cols = list(set([_v for k,v in nan_modes.items() for _v in v if 'c_mode_rtc' in k]))

100%|██████████| 11/11 [00:56<00:00,  5.17s/it]


In [8]:
celldyn.loc[celldyn.c_mode_rtc==0, reti_mode_cols] = np.nan

# Clean based on flags and limits

In [9]:
qc = QcControl(param_file="../external/CelldynParams.csv", 
               reference_file=None) 
celldyn_cleaned = pd.DataFrame(data=qc.fit_transform(celldyn), columns=celldyn.columns)

Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


# Scale celldyn measurements

In [ ]:
CellDynTrans = CellDynTrans(log_scale=['c_b_seg', 'c_b_neu', 'c_b_wbc'])
celldyn_cleaned_transformed = CellDynTrans.fit_transform(celldyn_cleaned)

In [11]:
celldyn_cleaned_transformed.reset_index().to_feather("T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_cleaned_transformed.feather")
celldyn_cleaned_transformed = celldyn_cleaned_transformed._to_pandas()

# Get full data frame

In [12]:
celldyn_cleaned_full_transformed = celldyn_cleaned_transformed.dropna(subset=meas_columns, axis=0)
celldyn_cleaned_full_transformed.reset_index().to_feather("T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_fullcase_cleaned_transformed.feather")

In [13]:
del celldyn
del celldyn_cleaned
del celldyn_cleaned_full_transformed
gc.collect()

0

# Impute

* [Fancy impute](https://pypi.org/project/fancyimpute/)
* [Auto impute](https://pypi.org/project/autoimpute/)
* [Skearn](https://scikit-learn.org/stable/modules/impute.html)
* [Statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.imputation.mice.MICE.html)
* [Miceforest](https://miceforest.readthedocs.io/en/latest/)

*  normalisatie --> KNN impute (hoeveel neighbors, wat voor distance..)
* https://arxiv.org/pdf/0805.4471.pdf: in fancyimpute; NuclearNormMinimization
* Iterative imputer met PMM : normalisation -> sklearn voor iterative imputer, autoimpute.imputations.series.pmm

In [15]:
# missing at random are the variables that are missing because we did not run in Reti modus
not_reti_mode_cols = [c for c in meas_columns if c not in reti_mode_cols]

idx_nreti = celldyn_cleaned_transformed[celldyn_cleaned_transformed.c_mode_rtc==0].index
idx_reti = celldyn_cleaned_transformed[celldyn_cleaned_transformed.c_mode_rtc==1].index

keep_fullcase = idx_nreti[celldyn_cleaned_transformed\
                            .loc[idx_nreti,
                                 not_reti_mode_cols]\
                            .isna().sum(axis=1)==0]
keep_reti_only_nans = idx_reti[celldyn_cleaned_transformed\
                            .loc[idx_reti,
                                 meas_columns]\
                            .isna().sum(axis=1)==0]
celldyn_fullcase_for_impute = celldyn_cleaned_transformed.loc[list(set(keep_fullcase).union(keep_reti_only_nans))]

In [16]:
from miceforest import ImputationKernel
from miceforest import mean_match_default

mean_match_default.set_mean_match_candidates(5)
impute_clf = ImputationKernel(
    celldyn_fullcase_for_impute[meas_columns],
    save_all_iterations=False,
    random_state=1283,
    mean_match_scheme=mean_match_default
    )
impute_clf.mice(iterations=5, verbose=True, n_estimators=100)
celldyn_fullcase_for_impute.loc[:, meas_columns] = impute_clf.complete_data()
gc.collect()


Initialized logger with name mice 1-5
Dataset 0
1  | c_b_pretc | c_b_pmic | c_b_pmac | c_b_rtcfcv | c_b_rtcfmn | c_b_rbcfcv | c_b_rbcfmn | c_b_rbcicv | c_b_rbcimn | c_b_mchr | c_b_mcvr | c_b_hdw | c_b_prp | c_b_phpr | c_b_phpo | c_b_irf | c_b_retc | c_b_mchcr
2  | c_b_pretc | c_b_pmic | c_b_pmac | c_b_rtcfcv | c_b_rtcfmn | c_b_rbcfcv | c_b_rbcfmn | c_b_rbcicv | c_b_rbcimn | c_b_mchr | c_b_mcvr | c_b_hdw | c_b_prp | c_b_phpr | c_b_phpo | c_b_irf | c_b_retc | c_b_mchcr
3  | c_b_pretc | c_b_pmic | c_b_pmac | c_b_rtcfcv | c_b_rtcfmn | c_b_rbcfcv | c_b_rbcfmn | c_b_rbcicv | c_b_rbcimn | c_b_mchr | c_b_mcvr | c_b_hdw | c_b_prp | c_b_phpr | c_b_phpo | c_b_irf | c_b_retc | c_b_mchcr
4  | c_b_pretc | c_b_pmic | c_b_pmac | c_b_rtcfcv | c_b_rtcfmn | c_b_rbcfcv | c_b_rbcfmn | c_b_rbcicv | c_b_rbcimn | c_b_mchr | c_b_mcvr | c_b_hdw | c_b_prp | c_b_phpr | c_b_phpo | c_b_irf | c_b_retc | c_b_mchcr
5  | c_b_pretc | c_b_pmic | c_b_pmac | c_b_rtcfcv | c_b_rtcfmn | c_b_rbcfcv | c_b_rbcfmn | c_b_rbcicv | 

0

In [17]:
from miceforest import ImputationKernel
from miceforest import mean_match_default

mean_match_default.set_mean_match_candidates(5)
impute_clf = ImputationKernel(
    celldyn_cleaned_transformed[meas_columns],
    save_all_iterations=False,
    random_state=1283,
    mean_match_scheme=mean_match_default
    )
impute_clf.mice(iterations=5, verbose=True, n_estimators=100)
celldyn_cleaned_transformed.loc[:, meas_columns] = impute_clf.complete_data()
gc.collect()

# impute_clft can be put in pipe :))
# [clean, scale, impute_clf]

Initialized logger with name mice 1-5
Dataset 0
1  | plt | c_b_pmone | c_b_plyme | c_b_hgb_usa | c_b_mch_usa | c_b_mone | c_b_lyme | c_b_mchc_usa | c_b_pct | c_b_wvf | c_b_pdw | c_b_mch | c_b_mchc | c_b_hb | c_b_rdw | c_b_rbco | c_b_mcv | c_b_rbci | c_b_ht | c_b_plti | c_b_nrbc | c_b_pnrbc | c_b_pbas | c_b_bas | c_b_pblst | c_b_blst | c_b_wbc | c_b_pvlym | c_b_plym | c_b_lym | c_b_vlym | c_b_mon | c_b_plto | c_b_pmon | c_b_pneu | c_b_pseg | c_b_pbnd | c_b_pig | c_b_bnd | c_b_ig | c_b_seg | c_b_neu | c_b_peos | c_b_eos | c_b_mpv | c_b_ppmn | c_b_pimn | c_b_picv | c_b_ppcv | c_b_licv | c_b_limn | c_b_lacv | c_b_lamn | c_b_ndcv | c_b_ndmn | c_b_namn | c_b_nacv | c_b_nimn | c_b_nicv | c_b_nfmn | c_b_nfcv | c_b_npmn | c_b_npcv | c_b_irf | c_b_pretc | c_b_retc | c_b_rtcfcv | c_b_rtcfmn | c_b_rbcicv | c_b_rbcfcv | c_b_rbcimn | c_b_rbcfmn | c_b_pmic | c_b_pmac | c_b_prp | c_b_hdw | c_b_mcvr | c_b_mchr | c_b_mchcr | c_b_phpr | c_b_phpo
2  | plt | c_b_pmone | c_b_plyme | c_b_hgb_usa | c_b_mch_us

0

In [ ]:
 # c_b_bnd: consider dropping -> sigmoid_log(v)
# c_b_ig: consider dropping -> sigmoid_log(v)
# c_b_vlym: consider dropping -> sigmoid_log(v)
# c_b_blst: consider dropping -> sigmoid_log(v)
# c_b_nrbc: consider dropping -> sigmoid_log(v)
# c_b_pnrbc: consider dropping -> sigmoid_log(v)
# c_b_pbnd: consider dropping -> sigmoid_log(v)
# c_b_pig: consider dropping -> sigmoid_log(v)
# c_b_pvlym: consider dropping -> sigmoid_log(v)
# c_b_pnrbc: consider dropping -> sigmoid_log(v)
# c_b_pblst: consider dropping -> sigmoid_log(v)


In [122]:
### ADD combinations after filtering and scaling
# CD4/CD8 ratio
# T-helper: CD4+
# Cytotoxic: CD8+
# T-inhibitor: CD4+, CD25+, FOXP3
# T-memory

combo_features = [(['mon','mone'],['lym', 'lyme', 'vlym']), 
                  (['bas', 'eos', 'neu'], ['seg', 'bnd', 'mon', 'vlym',
                                            'lym', 'lyme', 'nrbc']),
                (['pmon', 'pmone'], ['pvlym','plym','plyme']),
                (['pneu', 'pbas', 'peos'], ['pseg', 'pbnd', 'pmon', 
                                            'pvlym', 'plyme','plym',
                                            'pnrbc'])]
extra_features = 0
derived_columns = []
eps = 1
for t in combo_features:
    for left in t[0]:
        for right in t[1]:
            # add column with the ratios of the columns in the two lists
    	    
            if (right=='c_b_seg') & (left=='c_b_neu'):
                epsl=eps
            else:
                epsl=eps

            celldyn_cleaned_transformed[left+':'+right] = (np.log10(celldyn_cleaned_transformed['c_b_'+left]+epsl)+1)/(np.log10(celldyn_cleaned_transformed['c_b_'+right]+eps)+1)
            derived_columns.append(left+':'+right)
            extra_features += 1
print(f"We added {extra_features} extra features")
meas_columns = meas_columns+derived_columns
meas_names = ["_".join(c.split("_")[2:]) for c in meas_columns]

We added 54 extra features


In [ ]:
extra_features = 0
derived_columns = []
eps = 1
for t in combo_features:
    for left in t[0]:
        for right in t[1]:
            # add column with the ratios of the columns in the two lists
    	    
            if (right=='c_b_seg') & (left=='c_b_neu'):
                epsl=eps
            else:
                epsl=eps

            celldyn_fullcase_for_impute[left+':'+right] = (np.log10(celldyn_fullcase_for_impute['c_b_'+left]+epsl)+1)/(np.log10(celldyn_fullcase_for_impute['c_b_'+right]+eps)+1)
            derived_columns.append(left+':'+right)
            extra_features += 1
print(f"We added {extra_features} extra features")
meas_columns = meas_columns+derived_columns
meas_names = ["_".join(c.split("_")[2:]) for c in meas_columns]

In [123]:
remove_cols = ['pneu:pseg', 'pbas:pseg', 'peos:pseg', 'neu:seg']
celldyn_cleaned_transformed.drop(remove_cols, axis=1, inplace=True)
celldyn_fullcase_for_impute.drop(remove_cols, axis=1, inplace=True)

In [2]:
celldyn_fullcase_for_impute = pd.read_feather("T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_cleaned_transformed_imputed_MCAR.feather")
celldyn_cleaned_transformed = pd.read_feather("T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_cleaned_transformed_imputed_ALL.feather")

In [3]:
from recombinators import CellDynRecombinator
recombiner = CellDynRecombinator()

In [4]:
recombiner.fit(celldyn_fullcase_for_impute)
celldyn_fullcase_for_impute = recombiner.transform(celldyn_fullcase_for_impute)

recombiner.fit(celldyn_cleaned_transformed)
celldyn_cleaned_transformed = recombiner.transform(celldyn_cleaned_transformed)

Fitting


Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


Fitting


In [5]:
celldyn_fullcase_for_impute = celldyn_fullcase_for_impute._to_pandas()
celldyn_cleaned_transformed = celldyn_cleaned_transformed._to_pandas()

In [6]:
celldyn_fullcase_for_impute.reset_index().to_feather("T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_cleaned_transformed_imputed_MCAR_with_ratios.feather")
gc.collect()

celldyn_cleaned_transformed.reset_index().to_feather("T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_cleaned_transformed_imputed_ALL_with_ratios.feather")
gc.collect()

0